<a href="https://colab.research.google.com/github/donghui-0126/mini-project/blob/main/shoes-project/resell_regressor/bn_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.linear_model import LinearRegression
import warnings
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/kream_data/product_data_dpp.csv')

In [4]:
df.shape

(6103, 15)

In [5]:
df_nike = df[df['brand']=='Nike'][['brand', 'color1', 'color2', 'price_og', 'price_resell']]
df_jordan = df[df['brand']=='Jordan'][['brand', 'color1', 'color2', 'price_og', 'price_resell']]
df_adidas = df[df['brand']=='Adidas'][['brand', 'color1', 'color2', 'price_og', 'price_resell']]

In [6]:
print(df_nike.shape)
df_nike.head()

(1455, 5)


,brand,color1,color2,price_og,price_resell
0,Nike,WHITE,BLACK,129000,152800
1,Nike,WHITE,WHITE,139000,136200
2,Nike,FLAX,GUM,169000,173200
3,Nike,FLAX,FLAX-GUM LIGHT BROWN,184600,281400
4,Nike,LIGHTT IRON ORE,METALLIC SILVER,209000,258000


In [7]:
print(df_jordan.shape)
df_jordan.head()

(1376, 5)


,brand,color1,color2,price_og,price_resell
1455,Jordan,VARSITY RED,BLACK,209000,480800
1456,Jordan,WHITE,BLACK,139000,163600
1457,Jordan,WHITE,BLACK,249000,254400
1458,Jordan,BLACK,PHANTOM,189000,698000
1459,Jordan,SAIL,UNIVERSITY RED-RIDGEROCK,189000,1417800


In [8]:
df_nike['price_resell'].describe()

count    1.455000e+03
mean     2.563218e+05
std      2.945740e+05
min      4.560000e+04
25%      1.389000e+05
50%      1.900000e+05
75%      2.816000e+05
max      8.018000e+06
Name: price_resell, dtype: float64

In [9]:
df_jordan['price_resell'].describe()

count    1.376000e+03
mean     3.379037e+05
std      5.207507e+05
min      5.000000e+04
25%      1.740500e+05
50%      2.399000e+05
75%      3.340500e+05
max      9.676000e+06
Name: price_resell, dtype: float64

In [10]:
(df_nike['price_resell']//100000).value_counts()

1     668
2     367
3     137
0     111
4      62
5      27
6      25
9      15
8      12
7      11
11      5
10      4
14      2
13      2
22      2
17      1
12      1
20      1
25      1
80      1
Name: price_resell, dtype: int64

In [11]:
df_nike = df_nike.drop(df_nike[df_nike['price_resell']>8000000].index)

In [12]:
df_nike['price_class'] = np.where(df_nike['price_resell'] <1000000,\
                                  df_nike['price_resell']//100000,\
                                  10)

-----

In [13]:
((df_jordan['price_resell'])//100000).value_counts()

2     468
1     428
3     199
4      78
5      52
0      51
6      27
7      13
8      12
10      6
9       5
19      5
13      4
15      2
24      2
30      2
17      2
14      2
20      2
12      2
33      1
22      1
64      1
41      1
18      1
65      1
32      1
31      1
16      1
79      1
21      1
96      1
11      1
28      1
Name: price_resell, dtype: int64

In [14]:
df_jordan = df_jordan.drop(df_nike[df_nike['price_resell']>5000000].index)

In [15]:
df_jordan['price_class'] = np.where(df_jordan['price_resell'] <1100000,\
                                  df_jordan['price_resell']//100000,\
                                  13)

---------

In [16]:
def make_input_col(df_list):
    for df in df_list:
        df['input'] = df['brand'] + " | " + df['color1'] + " | " + df['color2']
        df['target'] = df['price_resell']/10000.0

make_input_col([df_nike, df_jordan, df_adidas])

In [17]:
def tokenize_input(df_list, pd_list):
    return_list = []

    for df, pd in zip(df_list, pd_list):
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(df['input'])
        word_index = tokenizer.word_index

        print(f"word_index의 크기: {len(word_index)}")

        # 문장을 토큰화 한다.
        tokenized_name = tokenizer.texts_to_sequences(df['input'])
        padded_name = pad_sequences(tokenized_name)

        return_list.append((padded_name))
    return return_list

df_nike_input_ = pd.DataFrame()
df_jordan_input_ = pd.DataFrame()
df_adidas_input_ = pd.DataFrame()

[df_nike_input, df_jordan_input, df_adidas_input] = tokenize_input([df_nike, df_jordan, df_adidas], [df_nike_input_, df_jordan_input_, df_adidas_input_])

word_index의 크기: 515
word_index의 크기: 389
word_index의 크기: 319


In [19]:
from sklearn.model_selection import train_test_split

In [20]:
nike_train_X, nike_test_X, nike_train_y, nike_test_y = train_test_split(df_nike_input, df_nike['target'], test_size=0.3, shuffle=True, stratify=df_nike['price_class'])
nike_valid_X, nike_test_X, nike_valid_y, nike_test_y = train_test_split(nike_test_X, nike_test_y, test_size=0.5)

jordan_train_X, jordan_test_X, jordan_train_y, jordan_test_y = train_test_split(df_jordan_input, df_jordan['target'], test_size=0.3, shuffle=True, stratify=df_jordan['price_class'])
jordan_valid_X, jordan_test_X, jordan_valid_y, jordan_test_y = train_test_split(jordan_test_X, jordan_test_y, test_size=0.5)

adidas_train_X, adidas_test_X, adidas_train_y, adidas_test_y = train_test_split(df_adidas_input, df_adidas['target'], test_size=0.3, shuffle=True)
adidas_valid_X, adidas_test_X, adidas_valid_y, adidas_test_y = train_test_split(adidas_test_X, adidas_test_y, test_size=0.5)

In [21]:
%cd /content/drive/MyDrive/kream_data/model

/content/drive/MyDrive/kream_data/model


In [22]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
mc_nike=ModelCheckpoint('nike_color_best_model.h5', monitor='val_loss', verbose=1, save_best_only=True)
mc_jordan=ModelCheckpoint('jordan_color_best_model.h5', monitor='val_loss', verbose=1, save_best_only=True)

In [24]:
# 임베딩 층에 입력될 단어의 수를 지정한다.
word_size = 1216 + 1


nike_color_model = keras.models.Sequential([
    # 임베딩층
    keras.layers.Embedding(word_size, word_size, input_length=13),
    # 회귀를 위해 임베딩층의 output값을 1차원의 array로 바꾸어준다.
    keras.layers.Flatten(),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(32,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(16,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(8,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4,activation='relu'),
    keras.layers.Dropout(0.5),

    keras.layers.Dense(2,activation='relu'),
    keras.layers.Dense(1)
 ])

nike_color_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005), loss='mse', metrics= ["mean_absolute_percentage_error"])

nike_color_model.fit(nike_train_X, nike_train_y, batch_size=16, epochs=300, validation_data = (nike_valid_X,nike_valid_y),  callbacks=[es, mc_nike])

Epoch 1/300
61/64 [===========================>..] - ETA: 0s - loss: 1073.3906 - mean_absolute_percentage_error: 99.0072
Epoch 1: val_loss improved from inf to 1305.03479, saving model to nike_color_best_model.h5
64/64 [==============================] - 13s 86ms/step - loss: 1055.3020 - mean_absolute_percentage_error: 98.9312 - val_loss: 1305.0348 - val_mean_absolute_percentage_error: 97.2716
Epoch 2/300
64/64 [==============================] - ETA: 0s - loss: 998.4100 - mean_absolute_percentage_error: 93.7764 
Epoch 2: val_loss improved from 1305.03479 to 1166.60352, saving model to nike_color_best_model.h5
64/64 [==============================] - 2s 25ms/step - loss: 998.4100 - mean_absolute_percentage_error: 93.7764 - val_loss: 1166.6035 - val_mean_absolute_percentage_error: 80.5261
Epoch 3/300
61/64 [===========================>..] - ETA: 0s - loss: 891.0835 - mean_absolute_percentage_error: 83.4300
Epoch 3: val_loss did not improve from 1166.60352
64/64 [==========================

In [25]:
model = keras.models.load_model('nike_color_best_model.h5', custom_objects=None, compile=True)

model.evaluate(nike_test_X, nike_test_y)

7/7 [==============================] - 0s 3ms/step - loss: 314.0764 - mean_absolute_percentage_error: 63.7500


[314.076416015625, 63.749996185302734]

In [26]:
# 임베딩 층에 입력될 단어의 수를 지정한다.
word_size = 967 + 1


jordan_color_model = keras.models.Sequential([
    # 임베딩층
    keras.layers.Embedding(word_size, word_size, input_length=13),
    # 회귀를 위해 임베딩층의 output값을 1차원의 array로 바꾸어준다.
    keras.layers.Flatten(),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(32,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(16,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(8,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4,activation='relu'),
    keras.layers.Dropout(0.5),

    keras.layers.Dense(2,activation='relu'),
    keras.layers.Dense(1)
 ])

jordan_color_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005), loss='mse', metrics= ["mean_absolute_percentage_error"])

jordan_color_model.fit(jordan_train_X, jordan_train_y, batch_size=16, epochs=300, validation_data = (jordan_valid_X,jordan_valid_y),  callbacks=[es, mc_jordan])

Epoch 1/300
61/61 [==============================] - ETA: 0s - loss: 4318.5278 - mean_absolute_percentage_error: 93.0274
Epoch 1: val_loss improved from inf to 3530.98755, saving model to jordan_color_best_model.h5
61/61 [==============================] - 9s 101ms/step - loss: 4318.5278 - mean_absolute_percentage_error: 93.0274 - val_loss: 3530.9875 - val_mean_absolute_percentage_error: 81.8324
Epoch 2/300
61/61 [==============================] - ETA: 0s - loss: 4183.7451 - mean_absolute_percentage_error: 86.1249
Epoch 2: val_loss improved from 3530.98755 to 3252.14624, saving model to jordan_color_best_model.h5
61/61 [==============================] - 2s 25ms/step - loss: 4183.7451 - mean_absolute_percentage_error: 86.1249 - val_loss: 3252.1462 - val_mean_absolute_percentage_error: 62.6255
Epoch 3/300
61/61 [==============================] - ETA: 0s - loss: 4010.4153 - mean_absolute_percentage_error: 85.9642
Epoch 3: val_loss did not improve from 3252.14624
61/61 [====================

In [27]:
model = keras.models.load_model('jordan_color_best_model.h5', custom_objects=None, compile=True)

model.evaluate(jordan_test_X, jordan_test_y)

7/7 [==============================] - 0s 3ms/step - loss: 517.5479 - mean_absolute_percentage_error: 76.4892


[517.5479125976562, 76.48915100097656]